In [1]:
import numpy as np
import torch
import time
import torch.nn as nn
import os

torch.set_num_threads(1)

from threadpoolctl import threadpool_limits


ModuleNotFoundError: No module named 'torch'

In [9]:
def perf_matmul(ashape, bshape):
    A = torch.rand(ashape)
    B = torch.rand(bshape)
    start = time.time()
    C = torch.matmul(A, B)
    print(C.size())
    print(time.time() - start)
    
def np_perf(ashape, bshape):
    A = np.random.randn(*ashape)
    B = np.random.randn(*bshape)
    start = time.time()
    with threadpool_limits(limits=1, user_api='blas'):
        C = A @ B
        print(C.shape)
    print(time.time() - start)
    
def save_to(base, x, y):
    np_input = x.detach().numpy()
    with open(os.path.join(base, 'dummy_in.dat'), 'wb') as f:
        np_input.tofile(f)

    np_output = y.detach().numpy()
    with open(os.path.join(base, 'dummy_out.dat'), 'wb') as f:
        np_output.tofile(f)

In [3]:
perf_matmul((1008, 64, 512), (512, 1536))

np_perf((1008, 64, 512), (512, 1536))

torch.Size([1008, 64, 1536])
2.458440065383911
(1008, 64, 1536)
5.642819404602051


In [4]:
perf_matmul((128, 1008, 1008), (128, 1008, 256))

np_perf((128, 1008, 1008), (128, 1008, 256))

torch.Size([128, 1008, 256])
1.9739768505096436
(128, 1008, 256)
3.0647637844085693


In [3]:
perf_matmul((64512, 512), (512, 512))

np_perf((64512, 512), (512, 512))

torch.Size([64512, 512])
0.7357625961303711
(64512, 512)
1.724242925643921


In [6]:
perf_matmul((1008, 64, 512), (512, 2048))

np_perf((1008, 64, 512), (512, 2048))

torch.Size([1008, 64, 2048])
2.8759877681732178
(1008, 64, 2048)
7.270247936248779


In [7]:
perf_matmul((1008, 64, 2048), (2048, 512))
np_perf((1008, 64, 2048), (2048, 512))

torch.Size([1008, 64, 512])
2.827690601348877
(1008, 64, 512)
7.1652514934539795


In [3]:
w = torch.nn.Linear(2, 3)

x = torch.rand((4, 2))

In [4]:
%%time

y = w(x)
y.size()

CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 1.69 ms


torch.Size([4, 3])

In [6]:
input_names = ["x"]
output_names = ["y"]
dummy_input = x
torch.onnx.export(w,
                  dummy_input,
                  "./gemm.onnx",
                  verbose=False,
                  input_names=input_names,
                  output_names=output_names)

In [13]:
save_to(".", x, y)

tensor([[-0.5996,  0.1297,  0.0316],
        [-0.9554,  0.3789,  0.3892],
        [-0.6218,  0.6133,  0.3626],
        [-0.6292,  0.5457,  0.3221]], grad_fn=<AddmmBackward0>)